<a href="https://colab.research.google.com/github/uniyeh/BioMedVis-Challenge-2024/blob/main/Copy_of_BioMedVis_Challenge_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BioMedVis Challenge 2024

## Package Installation to Set Up the Environment:

In [ ]:
pip install vitessce anywidget uvicorn starlette ome-zarr ome_types

## Accessing the Data to create configurations

In [ ]:
import dask.array as da
import zarr
import requests
import io
from ome_zarr.io import parse_url
import ome_types

In [ ]:
path = "https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0"
root = parse_url(path, mode="w")
store = root.store
daskArray = da.from_zarr(store, component="3") # t=0, high-res
daskArray

In [ ]:
import requests
response = requests.get("https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/OME/METADATA.ome.xml")
data = response.text
ome_xml = ome_types.from_xml(response.text.replace("Â",""))
channel_names = [c.name for c in ome_xml.images[0].pixels.channels]
# Listing all the Channels in the Volume
channel_names

In [ ]:
# Reading the Physical Size of the Sample
ome_xml.images[0].pixels.physical_size_x, ome_xml.images[0].pixels.physical_size_y, ome_xml.images[0].pixels.physical_size_z

In [ ]:
# Getting the data from Channel CD31:
data = daskArray[0,19,:,:,:].compute()
data.shape

# Visualization of volumetric imaging data


Importing the necessary libraries:

In [ ]:
from vitessce import (
    VitessceConfig,
    CoordinationLevel as CL,
    get_initial_coordination_scope_prefix,
)
from os.path import join
import ipywidgets

## Getting a first view of the Data:

In [ ]:
vc = VitessceConfig(schema_version="1.0.16", name="BioMedVis Challenge")
dataset = vc.add_dataset(name="Blood Vessel", uid="bv").add_file(
    url="https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0",
    file_type="image.ome-zarr"
)

spatial = vc.add_view("spatialBeta", dataset=dataset)
lc = vc.add_view("layerControllerBeta", dataset=dataset)

#Spatial Target X,Y,Z are used for translating the viewpoint to a certain point in the volume
#These coordiantes are given in the best resolution level
vc.link_views_by_dict([spatial, lc], {
    "spatialTargetZ": 0,
    "spatialTargetT": 0,
    "spatialZoom": -3.54,
    "spatialTargetX": 5230,
    "spatialTargetY": 2880,
    "spatialTargetZ": 0,
    "spatialRenderingMode": "3D",
    "imageLayer": CL([
      {
        "spatialTargetResolution": 3,
        "spatialLayerOpacity": 1.0,
        "spatialLayerVisible": True,
        "photometricInterpretation": "BlackIsZero",
        "imageChannel": CL([
          {
            "spatialTargetC": 0,
            "spatialChannelColor": [255, 255,255],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
          }])
      }
    ])
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "image"))

vc.layout(spatial | lc);
vw = vc.widget(js_package_version="3.4.5", remount_on_uid_change=False)
vw

# Visualizing Regions of Interest (ROI)

Based on the prior analysis we can define a viewpoint showing an area of interest based on prior analysis in: https://doi.org/10.1101/2023.11.10.566670

In [ ]:
vc = VitessceConfig(schema_version="1.0.16", name="BioMedVis Challenge")
dataset = vc.add_dataset(name="Blood Vessel", uid="bv").add_file(
    url="https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0",
    file_type="image.ome-zarr"
)

spatial = vc.add_view("spatialBeta", dataset=dataset)
lc = vc.add_view("layerControllerBeta", dataset=dataset)

#Spatial Target X,Y,Z are used for translating the viewpoint to a certain point in the volume
#These coordiantes are given in the best resolution level
vc.link_views_by_dict([spatial, lc], {
    "spatialTargetZ": 0,
    "spatialTargetT": 0,
    "spatialZoom": -1.1,
    "spatialTargetX": 2914,
    "spatialTargetY": 1267,
    "spatialTargetZ": 0,
    "spatialRenderingMode": "3D",
    "imageLayer": CL([
      {
        "spatialTargetResolution": 3,
        "spatialLayerOpacity": 1.0,
        "spatialLayerVisible": True,
        "photometricInterpretation": "BlackIsZero",
        "imageChannel": CL([
          {
            "spatialTargetC": 9,
            "spatialChannelColor": [255, 125, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [0,9486]
          },
          {
            "spatialTargetC": 19,
            "spatialChannelColor": [0, 255, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [666,21313]
          },
          {
            "spatialTargetC": 69,
            "spatialChannelColor": [255, 255, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [6,34]
          },
          {
            "spatialTargetC": 21,
            "spatialChannelColor": [255, 0, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [1638,36287]
          },
        ]),
      }
    ])
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "image"))

vc.layout(spatial | lc);

vw = vc.widget(js_package_version="3.4.5", remount_on_uid_change=False)
vw

In [ ]:
vc = VitessceConfig(schema_version="1.0.16", name="BioMedVis Challenge")
dataset = vc.add_dataset(name="Blood Vessel", uid="bv").add_file(
    url="https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0",
    file_type="image.ome-zarr"
)

spatial = vc.add_view("spatialBeta", dataset=dataset)
lc = vc.add_view("layerControllerBeta", dataset=dataset)

#Spatial Target X,Y,Z are used for translating the viewpoint to a certain point in the volume
#These coordiantes are given in the best resolution level
vc.link_views_by_dict([spatial, lc], {
    "spatialTargetZ": 0,
    "spatialTargetT": 0,
    "spatialZoom":  -1.91661,
    "spatialTargetX": 5254,
    "spatialTargetY": 2834,
    "spatialTargetZ": 0,
    "spatialRenderingMode": "3D",
    "imageLayer": CL([
      {
        "spatialTargetResolution": 3,
        "spatialLayerOpacity": 1.0,
        "spatialLayerVisible": True,
        "photometricInterpretation": "BlackIsZero",
        "imageChannel": CL([
          {
            "spatialTargetC": 9,
            "spatialChannelColor": [255, 125,0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [0,9486]
          },
          {
            "spatialTargetC": 19,
            "spatialChannelColor": [0, 0,255],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [888,6882]
          },
          {
            "spatialTargetC": 69,
            "spatialChannelColor": [255, 255, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [6,34]
          },
          {
            "spatialTargetC": 21,
            "spatialChannelColor": [255, 0,0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [0,6552]
          },
          {
            "spatialTargetC": 29,
            "spatialChannelColor": [255,0,255],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [475,2983]
          },
          {
            "spatialTargetC": 3,
            "spatialChannelColor": [0, 255,0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [1287,14825]
          },
        ]),
      }
    ])
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "image"))

vc.layout(spatial | lc);
vw = vc.widget(js_package_version="3.4.5", remount_on_uid_change=False)
vw

# Reading Visualization Parameters:

In [ ]:
vw.config["coordinationSpace"]["spatialTargetX"]

In [ ]:
vw.config["coordinationSpace"]["spatialTargetY"]

In [ ]:
vw.config["coordinationSpace"]["spatialTargetZ"]

In [ ]:
vw.config["coordinationSpace"]["spatialZoom"]